In [1]:
import openai
import gradio as gr
from dotenv import load_dotenv
import os
import requests
import json
import numpy as np

_ = load_dotenv()

In [2]:
chat_log = []

def predict(message, history):
    openai_messages = [
        {
            "role": "system",
            "content": (
                """
You are a Study Copilot: a warm, encouraging AI assistant whose mission is to help students fully understand academic concepts through flexible support modes. Your goal is to respond to where the student is in their learning journey and guide them patiently until they grasp the subject.

IMPORTANT: Before providing any educational content, you MUST follow this sequence:
1. Ensure the student has clearly selected one of the 4 learning modes (Test Mode, Tutor Mode, Initial Learning Mode, or Concept Mapper Mode)
2. Confirm they have specified a clear subject or topic they want to learn about
3. Assess their background and current knowledge level in the subject by asking about:
   - Their prior experience with the subject
   - Any specific concepts they already understand
   - Their comfort level with the material (beginner, intermediate, advanced)
   - Any specific goals they have for learning this subject

Only after gathering all this information should you proceed with tailored educational content. If any information is missing, politely ask for it before moving forward.

Your role includes:

1. **Intro + Discovery**
   - Greet the student warmly and explain your role.
   - Briefly introduce the 4 available learning modes:
     • Test Mode: Practice tests with feedback.
     • Tutor Mode: Targeted help to clarify confusing concepts.
     • Initial Learning Mode: Basic understanding with definitions and practice.
     • Concept Mapper Mode: Building structured study guides with examples.
   - Ask about the student's background and experience level in the subject.
   - Ask what they want to learn and if they'd like to upload files or use web searches.
   - Only ask **one question at a time** and do **not move on** until they feel confident.
   - Always ask if they have further questions before moving on.
2. **Test Mode**
   - Ask for the concept to test.
   - Ask how many questions they want (or choose appropriate number).
   - If situation requires it, give practice problems for student to work through (especially if stem)
   - Create a range of question difficulties, including critical thinking.
   - After the test, score and review incorrect answers one by one.
   - Ensure understanding before moving on.
   - Ask if they'd like another test or help with anything else.

3. **Tutor Mode**
   - Ask what they understand so far and what's confusing.
   - Explain concisely, using examples where helpful.
   - Answer follow-ups and check comprehension with single questions.
   - Provide real-world examples and explain how they connect.
   - Ask if they'd like help with anything else.

4. **Initial Learning Mode**
   - Start with simple questions about definitions/processes.
   - Offer practice problems after basic understanding is confirmed.
   - Review answers and walk through any the student struggled with.
   - Ask if they now feel confident or want to go deeper.

5. **Concept Mapper Mode**
   - Build a concise but detailed study guide showing connections between concepts.
   - Add examples and ask how they relate to the concept.
   - Ask questions based on the map one at a time, waiting for responses.
   - Ask if anything still needs clarification or if they'd like to continue.

Always end your messages with a single helpful question.
Be patient, concise, and encouraging.
"""
            )
        }
    ]
    openai_messages.extend(history)
    openai_messages.append({"role": "user", "content": message})
    
    client = openai.OpenAI()
    response = client.chat.completions.create(
        model='gpt-4',
        messages=openai_messages,
        temperature=1.0,
        stream=True
    )
    
    partial_message = ""
    for chunk in response:
        if hasattr(chunk.choices[0].delta, "content") and chunk.choices[0].delta.content is not None:
            partial_message += chunk.choices[0].delta.content
            yield partial_message
    
    return partial_message

initial_msg = """Hello! I'm your Study Copilot, an AI assistant designed to help you understand academic concepts through different learning modes.

To get started and provide you with the most helpful learning experience, I'll need some information:

1️⃣ **Select a learning mode**:
- **Test Mode**: Practice tests with feedback on your answers
- **Tutor Mode**: Targeted help to clarify confusing concepts
- **Initial Learning Mode**: Build basic understanding with definitions and practice
- **Concept Mapper Mode**: Create structured study guides with examples

2️⃣ **Choose a subject or topic** you want to learn about

3️⃣ Once you've shared these, I'll ask about your background knowledge in the subject to better tailor the content to your needs

What subject would you like to explore, and which learning mode would you prefer?"""

theme = gr.themes.Ocean(
    primary_hue="green",
    secondary_hue="violet",
    neutral_hue="indigo",
    font=["Inter", "sans-serif"],
    spacing_size="lg",
    radius_size="lg"
)

custom_css = """
.gradio-container {
    background-color: #ebf2fb !important;  
    color: #000000 !important;
}

#chatbot {
    background-color: #AAB7F8;
    padding: 16px;
    border-radius: 12px;
}
"""

with gr.Blocks(theme=theme, css=custom_css, title="Study Copilot") as demo:
    gr.Markdown("Welcome to **AIvy** 🌿")
    
    chatbot = gr.Chatbot(
        value=[{"role": "assistant", "content": initial_msg}],
        type="messages",
        height=600,
        elem_id="chatbot"
    )
    
    with gr.Row():
        msg = gr.Textbox(placeholder="Type your message here...", container=False)

    clear = gr.Button("Clear Chat")

    def user(user_message, history, file):
        history.append({"role": "user", "content": user_message})
        return "", history

    def bot(history):
        user_message = history[-1]["content"]
        bot_response = predict(user_message, history[:-1])
        history.append({"role": "assistant", "content": ""})
        for response in bot_response:
            history[-1]["content"] = response
            yield history
        chat_log.append({
        "user": user_message,
        "assistant": history[-1]["content"]
    })
    def save_chat_log(filename):
    
        with open(filename, "w") as f:
            json.dump(chat_log, f, indent=2)



    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )

    def clear_chat():
        return [{"role": "assistant", "content": initial_msg}]

    clear.click(clear_chat, None, chatbot)

demo.queue().launch(share=True)

/opt/jupyterhub/share/jupyter/venv/python3-12_comm4190/lib/python3.12/site-packages/gradio/utils.py:1017: UserWarning: Expected 3 arguments for function <function user at 0x7f7a3731f920>, received 2.
  warnings.warn(
/opt/jupyterhub/share/jupyter/venv/python3-12_comm4190/lib/python3.12/site-packages/gradio/utils.py:1021: UserWarning: Expected at least 3 arguments for function <function user at 0x7f7a3731f920>, received 2.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://169514ca64b8d6d519.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/opt/jupyterhub/share/jupyter/venv/python3-12_comm4190/lib/python3.12/site-packages/gradio/helpers.py:968: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")


In [13]:
save_chat_log("chatlog_test_mode.json")


In [14]:
save_chat_log("chatlog_tutor_mode.json")
 

In [15]:
save_chat_log("chatlog_initial_learning.json")


In [16]:
save_chat_log("chatlog_concept_mapper.json")

In [2]:
import shutil

shutil.rmtree('comm4190_S25_Using_LLMs_Blog')
